In [ ]:
#Importing the needed libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np
import os
import time

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from pathlib import Path
from collections import Counter
from sklearn.pipeline import Pipeline
from openpyxl import load_workbook


In [ ]:
#Reading the excel-document into a df
df = pd.read_excel('dataset_name.xlsx')

#dropping unwanted columns from the df
df2=df.drop(columns = ["Column 1", "Column 2"])

In [ ]:
#Splitting df into two based on ID
df2["ID"] = df2["ID"].astype(str)  #Making sure its a string
df_train = df2[df2["ID"].isin(["ID 1", "ID 2", "ID 3"])]
df_unseen = df2[df2["ID"] == "ID 4"]

In [ ]:
#defining which columns to keep av model input
X = df_train.drop(columns = ['Column 1', 'Column 2', 'Column 3'])

#defining which columns are the model outputs
y = df_train['Column 4']

#defining inputs and outputs for unseen dataset
X_unseen = df_unseen.drop(columns = ['Column 1', 'Column 2', 'Column 3')
y_unseen = df_unseen['Column 4']

In [ ]:
#Splitting the dataset using K-Fold cross-validation
k_fold = KFold(n_splits=10, random_state=66, shuffle=True)

#Displaying number of splits
k_fold.get_n_splits(X, y)

#Defining a pipeline for preprocessing and regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),                               
    ('rf', RandomForestRegressor(n_estimators=40,                              
                                 random_state=40))            
])

#Creating lists to store performance metrics from each fold 
mae_scores_train, mse_scores_train, r2_scores_train = [], [], []  
mae_scores_test, mse_scores_test, r2_scores_test = [], [], []     

#Performing cross-validation
for train_index, test_index in k_fold.split(X, y):
    #Splitting the data into training and test sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Skipping this fold if the test set has only one unique target value
    if len(np.unique(y_test)) == 1:
        print("Skipping this fold due to only one class in test set.")
        continue

    #Training the pipeline on the current fold's training data
    pipeline.fit(X_train, y_train)

    #Making predictions on both training and test data
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    #Calculating metrics for training data
    mae_train = mean_absolute_error(y_train, pred_train)
    mse_train = mean_squared_error(y_train, pred_train)
    r2_train = r2_score(y_train, pred_train)

    #Calculating metrics for test data
    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    #Storing metrics in the lists
    mae_scores_train.append(mae_train)
    mse_scores_train.append(mse_train)
    r2_scores_train.append(r2_train)

    mae_scores_test.append(mae_test)
    mse_scores_test.append(mse_test)
    r2_scores_test.append(r2_test)

    #Printing metrics for this fold
    print(f"Train -> MAE: {mae_train:.6f}, MSE: {mse_train:.6f}, R²: {r2_train:.6f}")
    print(f"Test  -> MAE: {mae_test:.6f}, MSE: {mse_test:.6f}, R²: {r2_test:.6f}\n")

#Printing average metrics across all folds
print("\nGjennomsnittlige resultater over alle folds:")
print(f"Train Mean MAE: {np.mean(mae_scores_train):.6f}, Test Mean MAE: {np.mean(mae_scores_test):.6f}")
print(f"Train Mean MSE: {np.mean(mse_scores_train):.6f}, Test Mean MSE: {np.mean(mse_scores_test):.6f}")
print(f"Train Mean R²: {np.mean(r2_scores_train):.6f}, Test Mean R²: {np.mean(r2_scores_test):.6f}")

In [ ]:
#Defining a pipeline that standardizes the data and applies Random Forest regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=66)) 
])

#Defining the hyperparameter grid for GridSearchCV
parameters_grid = {
    'rf__n_estimators': [300],               #Number of trees in the forest
    'rf__max_features': [5, 7],              #Number of features to consider 
    'rf__max_depth': [5, 7],                 #Maximum depth of the tree
    'rf__min_samples_split': [10, 15],       #Minimum number of samples required to split an internal node
    'rf__min_samples_leaf': [10, 15],        #Minimum number of samples required to be at a leaf node
    'rf__criterion': ['absolute_error'],     #Loss function to measure the quality of a split
}

#Performing grid search cross-validation using the pipeline and defined parameter grid
CV_RF = GridSearchCV(estimator=pipeline, param_grid=parameters_grid, cv=k_fold, scoring='r2', n_jobs=-1)

#Fitting the pipeline to the training data within the defined cross validation
CV_RF.fit(X_train, y_train)

#Printing the best combination of hyperparameters found
print('Best parameters: ', CV_RF.best_params_)

In [ ]:
#Standardizing training and test data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Extracting best hyperparameters from GridSearchCV and remove the 'rf__' pin front
best_params = {key.replace("rf__", ""): value for key, value in CV_RF.best_params_.items()}

#Initializing and training the Random Forest Regressor using the best parameters
RF_best = RandomForestRegressor(**best_params)
RF_best.fit(X_train_scaled, y_train)

#Making predictions on both training and test data
y_train_pred = RF_best.predict(X_train_scaled)
y_test_pred = RF_best.predict(X_test_scaled)

#Calculating and printing performance metrics for the training set
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

#Calculating and printing performance metrics for the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Train Mean Squared Error: {mse_train:.4f}")
print(f"Train R-squared: {r2_train:.4f}")
print(f"Test Mean Squared Error: {mse_test:.4f}")
print(f"Test R-squared: {r2_test:.4f}")


In [ ]:
#Scaling the unseen features using the same scaler fitted on training data
X_unseen_scaled = scaler.transform(X_unseen)

#Using the trained Random Forest model to predict target values for the unseen data
y_unseen_pred = RF_best.predict(X_unseen_scaled)

#Evaluating the model performance on the unseen dataset
mse_unseen = mean_squared_error(y_unseen, y_unseen_pred)
r2_unseen = r2_score(y_unseen, y_unseen_pred)

#Printing the results for interpretation
print(f'Unseen Mean Squared Error: {mse_unseen:.4f}')
print(f'Unseen R-squared: {r2_unseen:.4f}')

In [ ]:
#Defining a function to log model evaluation metrics and hyperparameters to an Excel-file of chosen name
def log_results(model_name, params, mse_train, r2_train, mse_test, r2_test, mse_unseen, r2_unseen, filename="Results.xlsx"):
    """ Logs model results in separate sheets within the same Excel file, ensuring appending works correctly. """

    #Creating DataFrame for this model run
    result = pd.DataFrame([{
        **params, 
        "MSE_Train": mse_train,
        "R2_Train": r2_train,
        "MSE_Test": mse_test,
        "R2_Test": r2_test,
        "MSE_Unseen": mse_unseen,
        "R2_Unseen": r2_unseen
    }])

    #Introducing a short delay to avoid file conflicts if running in multiple notebooks
    time.sleep(1)

    #Checking if the file exists
    file_exists = os.path.exists(filename)

    if not file_exists:
        #If file doesn't exist, creating a new one
        with pd.ExcelWriter(filename, engine="openpyxl", mode="w") as writer:
            result.to_excel(writer, sheet_name=model_name, index=False)
    else:
        #If file exists, loading it properly before appending
        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                #Reading existing sheet 
                try:
                    existing_df = pd.read_excel(filename, sheet_name=model_name, engine="openpyxl")
                    df_combined = pd.concat([existing_df, result], ignore_index=True)
                except (FileNotFoundError, ValueError):
                    df_combined = result  # If sheet does not exist, create it

                #Saveing results, ensuring correct appending
                df_combined.to_excel(writer, sheet_name=model_name, index=False)

        except PermissionError:
            #Printing error to warn user
            print(f"Error: Close the Excel file ({filename}) before running the script again.")
            
    #Printing a confirmation to ensure user results are logged
    print(f"Logged results for {model_name}: Train MSE={mse_train:.4f}, Test MSE={mse_test:.4f}, Unseen MSE={mse_unseen:.4f}")

In [ ]:
#Using the function to log results from RF regression model
log_results(
    model_name="RF_reg",
    params=best_params,
    mse_train=mse_train,  
    r2_train=r2_train,
    mse_test=mse_test,
    r2_test=r2_test,
    mse_unseen=mse_unseen,
    r2_unseen=r2_unseen
)